In [1]:
import numpy as np
from scipy.optimize import leastsq
import math
from utils import *
from scipy.spatial.transform import Rotation as R

In [48]:
def calc_matrix(params):
    theta__7, r__7, theta__8, r__8, alpha__8, d__8 = params
    return np.mat([[math.cos(theta__7) * math.cos(theta__8) - math.sin(theta__7) * math.cos(alpha__8) * math.sin(theta__8),-math.cos(theta__7) * math.sin(theta__8) - math.sin(theta__7) * math.cos(alpha__8) * math.cos(theta__8),math.sin(theta__7) * math.sin(alpha__8),math.sin(theta__7) * math.sin(alpha__8) * r__8 + math.cos(theta__7) * d__8],[math.sin(theta__7) * math.cos(theta__8) + math.cos(theta__7) * math.cos(alpha__8) * math.sin(theta__8),math.cos(theta__7) * math.cos(alpha__8) * math.cos(theta__8) - math.sin(theta__7) * math.sin(theta__8),-math.cos(theta__7) * math.sin(alpha__8),-math.cos(theta__7) * math.sin(alpha__8) * r__8 + d__8 * math.sin(theta__7)],[math.sin(alpha__8) * math.sin(theta__8),math.sin(alpha__8) * math.cos(theta__8),math.cos(alpha__8),math.cos(alpha__8) * r__8 + r__7],[0,0,0,1]])

def eq(params, res):
    mat = np.array(calc_matrix(params)-res)
    return mat.flatten()

def reduce_rotations(theta):
    if theta > np.pi:
        theta = theta % (2*np.pi)
    if theta < -np.pi:
        theta = theta % -(2*np.pi)
    return theta

In [5]:
def find_mdh_params_6d(mat, initial_guesses = (0, 0, 0, 0, 0, 0), verbose=False):
    if verbose:
        print(f"Target Matrix:\n")
        roundprint(np.array(mat))
    res = leastsq(eq, initial_guesses, args=mat)[0]
    
    
    if False:
        results = zip(["theta__7", "r__7", "theta__8", "r__8", "alpha__8", "d__8" ], res)
        print(f"Results:")
        for elem in results:
            print("\t"+elem[0]+" :\t\t"+f"{elem[1]:.2f}")
        print(f"\nCheck Matrix:")
        roundprint(np.array(calc_matrix(tuple(res))))
        
    # modulo for the angles
    res2 = res.copy()
    res2[0] = reduce_rotations(res2[0])
    res2[2] = reduce_rotations(res2[2])
    res2[4] = reduce_rotations(res2[4])
    
    if verbose:
        results = zip(["theta__7", "r__7", "theta__8", "r__8", "alpha__8", "d__8" ], res2)
        print(f"Results Modulo:")
        for elem in results:
            print("\t"+elem[0]+" :\t\t"+f"{elem[1]:.2f}")
        print(f"\nCheck Matrix Modulo:")
        roundprint(np.array(calc_matrix(tuple(res2))))
    print("residue")
    roundprint(np.array(calc_matrix(tuple(res2)))-mat)
    return res2

In [6]:
# random input matrix from angle axis
rvec = np.random.uniform(low=0.0, high=1, size=(3,))*np.random.uniform(low=0.0, high=np.pi, size=(1,))
tvec = np.random.uniform(low=0.0, high=0.2, size=(3,))
print(rvec, tvec)
target_matrix = H_rvec_tvec(rvec, tvec)

# find MDH values
res = find_mdh_params_6d(target_matrix, verbose=True)

[0.04209345 0.01971019 0.05293865] [0.08541471 0.01257875 0.19549853]
Target Matrix:

 0.998 -0.052  0.021  0.085 
 0.053  0.998 -0.042  0.013 
-0.019  0.043  0.999  0.195 
 0.000  0.000  0.000  1.000 


Results Modulo:
	theta__7 :		0.46
	r__7 :		-0.39
	theta__8 :		-0.41
	r__8 :		0.58
	alpha__8 :		0.05
	d__8 :		0.08

Check Matrix Modulo:
 0.998 -0.052  0.021  0.085 
 0.053  0.998 -0.042  0.013 
-0.019  0.043  0.999  0.195 
 0.000  0.000  0.000  1.000 


residue
 0.000  0.000  0.000  0.000 
 0.000  0.000  0.000  0.000 
 0.000  0.000  0.000  0.000 
 0.000  0.000  0.000  0.000 




In [7]:
#input matrix 
target_matrix = Trans(0, 0, 0.2)@Rx(np.pi/2)
# find MDH values
res = find_mdh_params_6d(target_matrix, verbose=True)

Target Matrix:

 1.000  0.000  0.000  0.000 
 0.000  0.000 -1.000  0.000 
 0.000  1.000  0.000  0.200 
 0.000  0.000  0.000  1.000 


Results Modulo:
	theta__7 :		-0.00
	r__7 :		0.20
	theta__8 :		-0.00
	r__8 :		0.00
	alpha__8 :		1.57
	d__8 :		-0.00

Check Matrix Modulo:
 1.000  0.000  0.000  0.000 
 0.000  0.000 -1.000  0.000 
 0.000  1.000  0.000  0.200 
 0.000  0.000  0.000  1.000 


residue
 0.000  0.000  0.000  0.000 
 0.000  0.000  0.000  0.000 
 0.000  0.000  0.000  0.000 
 0.000  0.000  0.000  0.000 




In [8]:
def rpy_trans_to_mat(roll, pitch, yaw, x, y, z):
    print(f"rpy:{[roll, pitch, yaw]}")
    print(f"xyz:{[x, y, z]}")
    print("\n")
    T = Trans(x,y,z)@Rz(yaw)@Ry(pitch)@Rx(roll)
    return T


In [9]:
# random input 
# v1 = np.random.uniform(low=0.0, high=1, size=(3,))
# v2 = np.random.uniform(low=0.0, high=0.2, size=(3,))
# rvec=np.round(rvec,2)
# tvec=np.round(tvec,2)

target_matrix = rpy_trans_to_mat(1, 0, 0, 0.1, 0, 0.3)

# find MDH values
res = find_mdh_params_6d(target_matrix, verbose=True)

rpy:[1, 0, 0]
xyz:[0.1, 0, 0.3]


Target Matrix:

 1.000  0.000  0.000  0.100 
 0.000  0.540 -0.841  0.000 
 0.000  0.841  0.540  0.300 
 0.000  0.000  0.000  1.000 


Results Modulo:
	theta__7 :		0.00
	r__7 :		0.30
	theta__8 :		-0.00
	r__8 :		0.00
	alpha__8 :		1.00
	d__8 :		0.10

Check Matrix Modulo:
 1.000  0.000  0.000  0.100 
 0.000  0.540 -0.841  0.000 
 0.000  0.841  0.540  0.300 
 0.000  0.000  0.000  1.000 


residue
 0.000  0.000  0.000  0.000 
 0.000  0.000  0.000  0.000 
 0.000  0.000  0.000  0.000 
 0.000  0.000  0.000  0.000 




In [44]:
# input from TU lab yaml file
qw = 0.38352032106776657
qx = 0.0003285916679779742
qy = -0.9235307610124668
qz = -0.0017288202125134702
    
x = -0.04245045620490106
y = 0.017897549834532466
z = 0.05654182395246343
    

quat = np.array([qx, qy, qz, qw])
rot = R.from_quat(quat)
rotmat = rot.as_matrix()
tvec = np.array([x, y, z])
mat = merge_H_transform(rotmat, tvec)
#roundprint(mat)

# find MDH values
res = find_mdh_params_6d(mat, verbose=True)

Target Matrix:

-0.706  0.001 -0.708 -0.042 
-0.002  1.000  0.003  0.018 
 0.708  0.003 -0.706  0.057 
 0.000  0.000  0.000  1.000 


Results Modulo:
	theta__7 :		-4.72
	r__7 :		0.10
	theta__8 :		4.71
	r__8 :		0.06
	alpha__8 :		-2.35
	d__8 :		0.02

Check Matrix Modulo:
-0.706  0.001 -0.708 -0.042 
-0.002  1.000  0.003  0.018 
 0.708  0.003 -0.706  0.057 
 0.000  0.000  0.000  1.000 


residue
 0.000  0.000  0.000  0.000 
 0.000  0.000  0.000  0.000 
 0.000  0.000  0.000  0.000 
 0.000  0.000  0.000  0.000 




In [45]:
res[0] = res[0]+2*np.pi
res[2] = res[2]-2*np.pi
res

array([ 1.56664439,  0.09891168, -1.57565986,  0.0600294 , -2.35437403,
        0.01772114])

In [46]:
resdeg = res.copy()
resdeg[0] = resdeg[0]*180/np.pi
resdeg[2] = resdeg[2]*180/np.pi
resdeg[4] = resdeg[4]*180/np.pi

In [47]:
resdeg

array([ 8.97621115e+01,  9.89116766e-02, -9.02786601e+01,  6.00294009e-02,
       -1.34895695e+02,  1.77211445e-02])